In [ ]:
from typing import Any, Tuple
import yaml
import numpy as np
from copy import deepcopy
from functools import reduce




In [ ]:
def sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    values = loader.construct_sequence(node.value[0][1])
    return {"type": "sweep", "values": values}


def coupled_sweep_constructor(
    loader: yaml.SafeLoader, node: yaml.nodes.MappingNode
) -> dict[str, Any]:
    target = node.value[0][1].value
    values = loader.construct_sequence(node.value[1][1])
    return {"target": target, "values": values, "type": "coupled-sweep"}


def get_loader():
    loader = yaml.SafeLoader
    loader.add_constructor("!sweep", sweep_constructor)
    loader.add_constructor("!coupled-sweep", coupled_sweep_constructor)
    return loader

In [ ]:
cfg = yaml.load(open("./config.yaml"), Loader=get_loader())

In [ ]:
cfg

In [ ]:
import copy


class Run:
    def __init__(self, config: dict):
        self.config = config
        self.run_configs = []
        sweep_targets = {}
        coupled_targets = {}
        self._cfg_helper(self.config, sweep_targets, coupled_targets)

        print(sweep_targets)
        print(coupled_targets)

        self.run_configs = self._construct_run_configs(sweep_targets, coupled_targets)
        num_elems = reduce(
            lambda x, y: x * y, [len(v["values"]) for _, v in sweep_targets.items()], 1
        )
        print(
            len(self.run_configs),
            num_elems,
            len(set([tuple(s) for s in self.run_configs])),
        )

    def _cfg_helper(
        self, cfg_node: dict, sweep_targets, coupled_targets
    ) -> Tuple[list[Any], list[Any]]:
        for key, node in cfg_node.items():
            if isinstance(node, dict) and "type" in node:
                if node["type"] == "sweep":
                    sweep_targets[key] = {"key": key, "values": node["values"]}
                elif node["type"] == "coupled-sweep":
                    coupled_targets[key] = {
                        "target": node["target"],
                        "values": node["values"],
                    }
            elif isinstance(node, dict):
                self._cfg_helper(node, sweep_targets, coupled_targets)
            else:
                continue  # we ignore leafs

        for k, v in coupled_targets.items():
            sweep_targets[v["target"]]["partner"] = v["values"]

        return sweep_targets

    def _element_helper(
        self, elements, current_list, possible_partner, all_lists, *args, i=0
    ):
        i += 1
        for k in range(len(current_list)):
            v = current_list[k]
            if possible_partner is not None:
                w = possible_partner[k]
                if i < len(all_lists):
                    self._element_helper(
                        elements,
                        all_lists[i][0],
                        all_lists[i][1],
                        all_lists,
                        *args,
                        v,
                        w,
                        i=i,
                    )
                else:
                    self._element_helper(
                        elements,
                        all_lists[i][0],
                        all_lists[i][1],
                        all_lists,
                        *args,
                        v,
                        i=i,
                    )
            else:
                elements.append([*args, v])

    def _construct_run_configs(self, sweep_targets, coupled_targets):
        # make list of tuple values
        keys = list(sweep_targets.keys())
        lists = [
            (
                sweep_targets[k]["values"],
                sweep_targets[k]["partner"] if "partner" in sweep_targets[k] else None,
            )
            for k in keys
        ]
        i = 0
        elements = []
        self._element_helper(elements, lists[i][0], lists[i][1], lists, i=i)

        configs = [copy.deepcopy(self.config) for _ in elements]
        
        for cfg, vals in zip(configs, elements):
            for (i, k) in enumerate(keys): 
                if sweep_targets[k]["partner"] is not None: 
                    # find the node in the config to update
                    # I require the path to the sweep node for this
                    pass 
                else: 
                    pass 

        return elements

In [ ]:
runner = Run(cfg)